In [10]:
import pandas as pd
import numpy as np

In [88]:
df = pd.read_csv(
    filepath_or_buffer="dataset/netflix_titles.csv"
)

In [89]:
# General information about dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [90]:
# Just for future.
df['date_added'] = pd.to_datetime(
    df['date_added'], 
    format='mixed'
)

### Filling with `Unknown` NaN Values

I will fill with "Unknown" `cast` and `country`. Not too much missing information.Moreover, I am also filling with "Not Rated" `rating` column.

Columns such as `director`, `date_added` and `duration` also will be filled with `Unknown`. It will not change too much information in dataset, yet the dataset will not be in `NaN` values

In [91]:
df['cast'] = df['cast'].fillna('Unknown')
df['country'] = df['country'].fillna('Unknown')
df['date_added'] = df['date_added'].fillna("Unknown")
df['director'] = df['director'].fillna("Unknown")
df['duration'] = df['duration'].fillna("Unknown")
df['rating'] = df['rating'].fillna('Not Rated')

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      8807 non-null   object
 4   cast          8807 non-null   object
 5   country       8807 non-null   object
 6   date_added    8807 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8807 non-null   object
 9   duration      8807 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


### Dropping `description` Column

It seems that we are not going to work with descriptions, thus, I am dropping it.

In [93]:
df = df.drop(columns='description', axis=0)

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      8807 non-null   object
 4   cast          8807 non-null   object
 5   country       8807 non-null   object
 6   date_added    8807 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8807 non-null   object
 9   duration      8807 non-null   object
 10  listed_in     8807 non-null   object
dtypes: int64(1), object(10)
memory usage: 757.0+ KB


### Working with `rating` column

In [95]:
df['rating'].unique()

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', '74 min', '84 min', '66 min', 'NR',
       'Not Rated', 'TV-Y7-FV', 'UR'], dtype=object)

In [96]:
df['rating'].value_counts()

rating
TV-MA        3207
TV-14        2160
TV-PG         863
R             799
PG-13         490
TV-Y7         334
TV-Y          307
PG            287
TV-G          220
NR             80
G              41
TV-Y7-FV        6
Not Rated       4
NC-17           3
UR              3
66 min          1
74 min          1
84 min          1
Name: count, dtype: int64

'74 min', '84 min' and '66 min' obviously are not ratings. Let's replace them with "Not Rated".

Furthermore, `UR` (Unrated), `NR` (Not Rated), and "Not Rated" all mean essentially the same thing. Thus, I can replace UR and NR with just "Not Rated"

In [97]:
duration_ratings = ['66 min', '74 min', '84 min']
df.loc[df['rating'].isin(duration_ratings), 'rating'] = 'Not Rated'

not_rated = ['UR', 'NR']
df.loc[df['rating'].isin(not_rated), 'rating'] = 'Not Rated'

In [98]:
df['rating'].value_counts()

rating
TV-MA        3207
TV-14        2160
TV-PG         863
R             799
PG-13         490
TV-Y7         334
TV-Y          307
PG            287
TV-G          220
Not Rated      90
G              41
TV-Y7-FV        6
NC-17           3
Name: count, dtype: int64

### Working with `country` column

In [99]:
df['country'].value_counts()

country
United States                                    2818
India                                             972
Unknown                                           831
United Kingdom                                    419
Japan                                             245
                                                 ... 
Russia, Spain                                       1
Croatia, Slovenia, Serbia, Montenegro               1
Japan, Canada                                       1
United States, France, South Korea, Indonesia       1
Canada, Mexico, Germany, South Africa               1
Name: count, Length: 749, dtype: int64

Those combined countries (like "Russia, Spain", "Croatia, Slovenia, Serbia, Montenegro") are problematic for analysis. For this reason, I will just keep first country as a primary country.

In [100]:
def take_first_country(source: str) -> str:
    source = source.split(",")
    return source[0]

In [101]:
take_first_country("Croatia, Slovenia, Serbia, Montenegro")

'Croatia'

In [102]:
df['country'] = df['country'].apply(func=take_first_country)

In [103]:
df['country'].value_counts()

country
United States     3211
India             1008
Unknown            831
United Kingdom     628
Canada             271
                  ... 
West Germany         1
Slovenia             1
Guatemala            1
Jamaica              1
Somalia              1
Name: count, Length: 87, dtype: int64

### Working with `listed_in` column

Why am I going to remove "International TV Shows/Movie" label from "listed_in"?"International" doesn't add meaningful information since Netflix is a global platform by default. The label doesn't describe the actual genre/content type (comedy, drama, thriller, etc.). A TV show being "International" doesn't tell you what kind of show it actually is. Some entries have it, some don't, creating inconsistency. 

In [104]:
def remove_label(source: str) -> None:
    source = source \
                  .replace("International TV Shows", "") \
                  .replace("International Movies", "") \
                  .replace(", ,", ",") \
                  .strip(", ")
    return source

In [105]:
df['listed_in'] = df['listed_in'].apply(remove_label)

In [106]:
df['listed_in'].value_counts(())

listed_in
Documentaries                                       545
Dramas                                              500
Dramas, Independent Movies                          352
Stand-Up Comedy                                     334
Comedies, Dramas                                    303
                                                   ... 
Independent Movies                                    1
Classic & Cult TV, TV Horror, TV Mysteries            1
Dramas, Horror Movies, Sci-Fi & Fantasy               1
British TV Shows, TV Dramas, TV Sci-Fi & Fantasy      1
Cult Movies, Dramas, Thrillers                        1
Name: count, Length: 421, dtype: int64

Besides this, I will define brand-new column for primary genres, assuming that first one in the combination is the primary one. 

In [107]:
def extract_primary_genre(source: str) -> str:
    source = source.split(",")
    return source[0]

In [108]:
df['primary_genre'] = df['listed_in'].apply(extract_primary_genre)

In [109]:
df['primary_genre'].value_counts().head()

primary_genre
Dramas                      1600
Comedies                    1210
Action & Adventure           859
Documentaries                829
Children & Family Movies     605
Name: count, dtype: int64

### Splitting Dataset

At this point, I suppose I have to split dataset into 2 distinct tables.First one will be about TV Shows, whereas second one will be on Movies  

In [118]:
tv_shows = df[df['type'] == 'TV Show']
movies = df[df['type'] == 'Movie']

In [119]:
tv_shows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2676 entries, 1 to 8803
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   show_id        2676 non-null   object
 1   type           2676 non-null   object
 2   title          2676 non-null   object
 3   director       2676 non-null   object
 4   cast           2676 non-null   object
 5   country        2676 non-null   object
 6   date_added     2676 non-null   object
 7   release_year   2676 non-null   int64 
 8   rating         2676 non-null   object
 9   duration       2676 non-null   object
 10  listed_in      2676 non-null   object
 11  primary_genre  2676 non-null   object
dtypes: int64(1), object(11)
memory usage: 271.8+ KB


In [121]:
tv_shows['director'].value_counts()

director
Unknown                   2446
Alastair Fothergill          3
Rob Seidenglanz              2
Ken Burns                    2
Stan Lathan                  2
                          ... 
Bumpy                        1
Michael Samuels              1
Mark Tonderai                1
Ken Burns, Lynn Novick       1
Michael Cumming              1
Name: count, Length: 223, dtype: int64

The column(`director`) nearly useless for most analyses. Such high missingness makes statistical analysis unreliable. Besides this, by dropping this column, I will simplify my dataset without losing valuable information

In [122]:
tv_shows = tv_shows.drop(columns='director', axis=0)

In [123]:
tv_shows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2676 entries, 1 to 8803
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   show_id        2676 non-null   object
 1   type           2676 non-null   object
 2   title          2676 non-null   object
 3   cast           2676 non-null   object
 4   country        2676 non-null   object
 5   date_added     2676 non-null   object
 6   release_year   2676 non-null   int64 
 7   rating         2676 non-null   object
 8   duration       2676 non-null   object
 9   listed_in      2676 non-null   object
 10  primary_genre  2676 non-null   object
dtypes: int64(1), object(10)
memory usage: 250.9+ KB


In [125]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6131 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   show_id        6131 non-null   object
 1   type           6131 non-null   object
 2   title          6131 non-null   object
 3   director       6131 non-null   object
 4   cast           6131 non-null   object
 5   country        6131 non-null   object
 6   date_added     6131 non-null   object
 7   release_year   6131 non-null   int64 
 8   rating         6131 non-null   object
 9   duration       6131 non-null   object
 10  listed_in      6131 non-null   object
 11  primary_genre  6131 non-null   object
dtypes: int64(1), object(11)
memory usage: 622.7+ KB


In [126]:
movies['director'].value_counts()

director
Unknown                    188
Rajiv Chilaka               19
Raúl Campos, Jan Suter      18
Suhas Kadav                 16
Marcus Raboy                15
                          ... 
Mu Chu                       1
Chandra Prakash Dwivedi      1
Majid Al Ansari              1
Peter Hewitt                 1
Haile Gerima                 1
Name: count, Length: 4355, dtype: int64

### Saving Splitted Datasets

In [129]:
tv_shows = tv_shows.drop(columns="type")
movies = movies.drop(columns="type")

In [130]:
tv_shows.to_csv(
    path_or_buf="dataset/tv_shows.csv",
    index=False,
    sep=",",
    encoding="utf-8",
    header=True
)

In [131]:
movies.to_csv(
    path_or_buf="dataset/movies.csv",
    index=False,
    sep=",",
    encoding="utf-8",
    header=True
)